In [1]:
!pip install -U gdown  # 설치 안되어 있다면 먼저 설치
!gdown 1bDf8OCVlZfJ6XauPIzZYfVkQhj7BMiFJ

# !gdown --folder 1miYmYG8DwNQHMamcfBwJdzISCgH4C1nL


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Downloading...
From (original): https://drive.google.com/uc?id=1bDf8OCVlZfJ6XauPIzZYfVkQhj7BMiFJ
From (redirected): https://drive.google.com/uc?id=1bDf8OCVlZfJ6XauPIzZYfVkQhj7BMiFJ&confirm=t&uuid=8c0df92e-1b89-445a-8a35-9b686210df51
To: /workspace/vlora.zip
100%|████████████████████████████████████████| 717M/717M [00:09<00:00, 73.5MB/s]


In [4]:
# from google.colab import drive
#drive.mount('/content/drive')

# !cp -r "/content/drive/MyDrive/vlora" ./myfolder
!unzip -q vlora.zip 

/bin/bash: line 1: unzip: command not found


In [5]:
import zipfile
import os

zip_path = "vlora.zip"
extract_dir = "vlora"

# 디렉토리 없으면 생성
os.makedirs(extract_dir, exist_ok=True)

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [9]:
print('done')

done


In [7]:
!pip install diffsynth easydict decord -q
import argparse
from easydict import EasyDict as ed


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!pip install "huggingface_hub[cli]"
!mkdir Wan2.1-T2V-1.3B
!huggingface-cli download Wan-AI/Wan2.1-T2V-1.3B --local-dir ./Wan2.1-T2V-1.3B 


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 22 files:   0%|                                 | 0/22 [00:00<?, ?it/s]Downloading 'assets/data_for_diff_stage.jpg' to 'Wan2.1-T2V-1.3B/.cache/huggingface/download/assets/UCVN8O248kjCIp8i1TvUXWfm-q4=.59aec08409f2d46b0e640e4e120dc7cca52c08c3de56d026602dbcff1ebf241a.incomplete'
Still waiting to acquire lock on Wan2.1-T2V-1.3B/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on Wan2.1-T2V-1.3B/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on Wan2.1-T2V-1.3B/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on Wan2.1-T2V-1.3B/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Still waiting to acquire lock on Wan2.1-T2V-1.3B/.cache/huggingface/.gitignore.lo

In [11]:
import types
from diffsynth.models import ModelManager
from diffsynth.models.wan_video_dit import WanModel
from diffsynth.models.wan_video_text_encoder import WanTextEncoder
from diffsynth.models.wan_video_vae import WanVideoVAE
from diffsynth.models.wan_video_image_encoder import WanImageEncoder
from diffsynth.schedulers.flow_match import FlowMatchScheduler
from diffsynth.pipelines.base import BasePipeline
from diffsynth.prompters import WanPrompter
import torch, os
from einops import rearrange
import numpy as np
from PIL import Image
from tqdm import tqdm
from typing import Optional

from diffsynth.vram_management import enable_vram_management, AutoWrappedModule, AutoWrappedLinear
from diffsynth.models.wan_video_text_encoder import T5RelativeEmbedding, T5LayerNorm
from diffsynth.models.wan_video_dit import RMSNorm, sinusoidal_embedding_1d
from diffsynth.models.wan_video_vae import RMS_norm, CausalConv3d, Upsample
from diffsynth.models.wan_video_motion_controller import WanMotionControllerModel



class WanVideoPipeline(BasePipeline):

    def __init__(self, device="cuda", torch_dtype=torch.float16, tokenizer_path=None):
        super().__init__(device=device, torch_dtype=torch_dtype)
        self.scheduler = FlowMatchScheduler(shift=5, sigma_min=0.0, extra_one_step=True)
        self.prompter = WanPrompter(tokenizer_path=tokenizer_path)
        self.text_encoder: WanTextEncoder = None
        self.image_encoder: WanImageEncoder = None
        self.dit: WanModel = None
        self.vae: WanVideoVAE = None
        self.motion_controller: WanMotionControllerModel = None
        self.model_names = ['text_encoder', 'dit', 'vae', 'image_encoder', 'motion_controller']
        self.height_division_factor = 16
        self.width_division_factor = 16
        self.use_unified_sequence_parallel = False


    def enable_vram_management(self, num_persistent_param_in_dit=None):
        dtype = next(iter(self.text_encoder.parameters())).dtype
        enable_vram_management(
            self.text_encoder,
            module_map = {
                torch.nn.Linear: AutoWrappedLinear,
                torch.nn.Embedding: AutoWrappedModule,
                T5RelativeEmbedding: AutoWrappedModule,
                T5LayerNorm: AutoWrappedModule,
            },
            module_config = dict(
                offload_dtype=dtype,
                offload_device="cpu",
                onload_dtype=dtype,
                onload_device="cpu",
                computation_dtype=self.torch_dtype,
                computation_device=self.device,
            ),
        )
        dtype = next(iter(self.dit.parameters())).dtype
        enable_vram_management(
            self.dit,
            module_map = {
                torch.nn.Linear: AutoWrappedLinear,
                torch.nn.Conv3d: AutoWrappedModule,
                torch.nn.LayerNorm: AutoWrappedModule,
                RMSNorm: AutoWrappedModule,
            },
            module_config = dict(
                offload_dtype=dtype,
                offload_device="cpu",
                onload_dtype=dtype,
                onload_device=self.device,
                computation_dtype=self.torch_dtype,
                computation_device=self.device,
            ),
            max_num_param=num_persistent_param_in_dit,
            overflow_module_config = dict(
                offload_dtype=dtype,
                offload_device="cpu",
                onload_dtype=dtype,
                onload_device="cpu",
                computation_dtype=self.torch_dtype,
                computation_device=self.device,
            ),
        )
        dtype = next(iter(self.vae.parameters())).dtype
        enable_vram_management(
            self.vae,
            module_map = {
                torch.nn.Linear: AutoWrappedLinear,
                torch.nn.Conv2d: AutoWrappedModule,
                RMS_norm: AutoWrappedModule,
                CausalConv3d: AutoWrappedModule,
                Upsample: AutoWrappedModule,
                torch.nn.SiLU: AutoWrappedModule,
                torch.nn.Dropout: AutoWrappedModule,
            },
            module_config = dict(
                offload_dtype=dtype,
                offload_device="cpu",
                onload_dtype=dtype,
                onload_device=self.device,
                computation_dtype=self.torch_dtype,
                computation_device=self.device,
            ),
        )
        if self.image_encoder is not None:
            dtype = next(iter(self.image_encoder.parameters())).dtype
            enable_vram_management(
                self.image_encoder,
                module_map = {
                    torch.nn.Linear: AutoWrappedLinear,
                    torch.nn.Conv2d: AutoWrappedModule,
                    torch.nn.LayerNorm: AutoWrappedModule,
                },
                module_config = dict(
                    offload_dtype=dtype,
                    offload_device="cpu",
                    onload_dtype=dtype,
                    onload_device="cpu",
                    computation_dtype=dtype,
                    computation_device=self.device,
                ),
            )
        if self.motion_controller is not None:
            dtype = next(iter(self.motion_controller.parameters())).dtype
            enable_vram_management(
                self.motion_controller,
                module_map = {
                    torch.nn.Linear: AutoWrappedLinear,
                },
                module_config = dict(
                    offload_dtype=dtype,
                    offload_device="cpu",
                    onload_dtype=dtype,
                    onload_device="cpu",
                    computation_dtype=dtype,
                    computation_device=self.device,
                ),
            )
        self.enable_cpu_offload()


    def fetch_models(self, model_manager: ModelManager):
        text_encoder_model_and_path = model_manager.fetch_model("wan_video_text_encoder", require_model_path=True)
        if text_encoder_model_and_path is not None:
            self.text_encoder, tokenizer_path = text_encoder_model_and_path
            self.prompter.fetch_models(self.text_encoder)
            self.prompter.fetch_tokenizer(os.path.join(os.path.dirname(tokenizer_path), "google/umt5-xxl"))
        self.dit = model_manager.fetch_model("wan_video_dit")
        self.vae = model_manager.fetch_model("wan_video_vae")
        self.image_encoder = model_manager.fetch_model("wan_video_image_encoder")
        self.motion_controller = model_manager.fetch_model("wan_video_motion_controller")

    @staticmethod
    def from_model_manager(model_manager: ModelManager, torch_dtype=None, device=None, use_usp=False):
        if device is None: device = model_manager.device
        if torch_dtype is None: torch_dtype = model_manager.torch_dtype
        pipe = WanVideoPipeline(device=device, torch_dtype=torch_dtype)
        pipe.fetch_models(model_manager)
        return pipe
        # if use_usp:
        #     from xfuser.core.distributed import get_sequence_parallel_world_size
        #     from ..distributed.xdit_context_parallel import usp_attn_forward, usp_dit_forward

        #     for block in pipe.dit.blocks:
        #         block.self_attn.forward = types.MethodType(usp_attn_forward, block.self_attn)
        #     pipe.dit.forward = types.MethodType(usp_dit_forward, pipe.dit)
        #     pipe.sp_size = get_sequence_parallel_world_size()
        #     pipe.use_unified_sequence_parallel = True
        # return pipe



    def denoising_model(self):
        return self.dit


    def encode_prompt(self, prompt, positive=True):
        prompt_emb = self.prompter.encode_prompt(prompt, positive=positive, device=self.device)
        return {"context": prompt_emb}


    def encode_image(self, image, pseudo_video_path=None, mask_video_path=None, end_image=None, num_frames=81, height=480, width=832):
        """
        Encode images and videos for I2V inference
        Args:
            image: First frame image (PIL Image)
            pseudo_video_path: Pseudo video file path (new mode)
            mask_video_path: Mask video file path (new mode)
            end_image: End frame image (original mode)
            num_frames: Number of frames
            height, width: Video dimensions
        """
        # New mode: Use pseudo video and mask video
        # Original mode: use end_image (backward compatibility)

        image = self.preprocess_image(image.resize((width, height))).to(self.device)
        clip_context = self.image_encoder.encode_image([image])
        msk = torch.ones(1, num_frames, height//8, width//8, device=self.device)
        msk[:, 1:] = 0
        if end_image is not None:
            end_image = self.preprocess_image(end_image.resize((width, height))).to(self.device)
            vae_input = torch.concat([image.transpose(0,1), torch.zeros(3, num_frames-2, height, width).to(image.device), end_image.transpose(0,1)],dim=1)
            msk[:, -1:] = 1
        else:
            vae_input = torch.concat([image.transpose(0, 1), torch.zeros(3, num_frames-1, height, width).to(image.device)], dim=1)

        msk = torch.concat([torch.repeat_interleave(msk[:, 0:1], repeats=4, dim=1), msk[:, 1:]], dim=1)
        msk = msk.view(1, msk.shape[1] // 4, 4, height//8, width//8)
        msk = msk.transpose(1, 2)[0]

        y = self.vae.encode([vae_input.to(dtype=self.torch_dtype, device=self.device)], device=self.device)[0]
        y = torch.concat([msk, y])
        y = y.unsqueeze(0)
        clip_context = clip_context.to(dtype=self.torch_dtype, device=self.device)
        y = y.to(dtype=self.torch_dtype, device=self.device)
        return {"clip_feature": clip_context, "y": y}


    def encode_control_video(self, control_video, tiled=True, tile_size=(34, 34), tile_stride=(18, 16)):
        control_video = self.preprocess_images(control_video)
        # preprocess_images returns tensor list, each may be [1, 3, H, W], need to remove batch dimension
        control_video = [tensor.squeeze(0) if tensor.dim() == 4 else tensor for tensor in control_video]
        control_video = torch.stack(control_video, dim=1)  # [3, T, H, W]
        control_video = control_video.to(dtype=self.torch_dtype, device=self.device)  # [3, T, H, W] - VAE expects this format
        latents = self.encode_video(control_video, tiled=tiled, tile_size=tile_size, tile_stride=tile_stride).to(dtype=self.torch_dtype, device=self.device)
        return latents


    def prepare_controlnet_kwargs(self, control_video, num_frames, height, width, clip_feature=None, y=None, tiled=True, tile_size=(34, 34), tile_stride=(18, 16)):
        if control_video is not None:
            control_latents = self.encode_control_video(control_video, tiled=tiled, tile_size=tile_size, tile_stride=tile_stride)
            if clip_feature is None or y is None:
                clip_feature = torch.zeros((1, 257, 1280), dtype=self.torch_dtype, device=self.device)
                y = torch.zeros((1, 16, (num_frames - 1) // 4 + 1, height//8, width//8), dtype=self.torch_dtype, device=self.device)
            else:
                y = y[:, -16:]
            y = torch.concat([control_latents, y], dim=1)
        return {"clip_feature": clip_feature, "y": y}


    def tensor2video(self, frames):
        frames = rearrange(frames, "C T H W -> T H W C")
        frames = ((frames.float() + 1) * 127.5).clip(0, 255).cpu().numpy().astype(np.uint8)
        frames = [Image.fromarray(frame) for frame in frames]
        return frames


    def prepare_extra_input(self, latents=None):
        return {}


    def encode_video(self, input_video, tiled=True, tile_size=(34, 34), tile_stride=(18, 16)):
        latents = self.vae.encode(input_video, device=self.device, tiled=tiled, tile_size=tile_size, tile_stride=tile_stride)
        return latents


    def decode_video(self, latents, tiled=True, tile_size=(34, 34), tile_stride=(18, 16)):
        frames = self.vae.decode(latents, device=self.device, tiled=tiled, tile_size=tile_size, tile_stride=tile_stride)
        return frames


    def prepare_unified_sequence_parallel(self):
        return {"use_unified_sequence_parallel": self.use_unified_sequence_parallel}


    def prepare_motion_bucket_id(self, motion_bucket_id):
        motion_bucket_id = torch.Tensor((motion_bucket_id,)).to(dtype=self.torch_dtype, device=self.device)
        return {"motion_bucket_id": motion_bucket_id}


    @torch.no_grad()
    def __call__(
        self,
        prompt,
        negative_prompt="",
        input_image=None,
        pseudo_video_path=None,
        mask_video_path=None,
        end_image=None,
        input_video=None,
        control_video=None,
        denoising_strength=1.0,
        seed=None,
        rand_device="cpu",
        height=480,
        width=832,
        num_frames=81,
        cfg_scale=5.0,
        num_inference_steps=50,
        sigma_shift=5.0,
        motion_bucket_id=None,
        tiled=True,
        tile_size=(30, 52),
        tile_stride=(15, 26),
        tea_cache_l1_thresh=None,
        tea_cache_model_id="",
        progress_bar_cmd=tqdm,
        progress_bar_st=None,
    ):
        # Parameter check
        height, width = self.check_resize_height_width(height, width)
        if num_frames % 4 != 1:
            num_frames = (num_frames + 2) // 4 * 4 + 1
            print(f"Only `num_frames % 4 != 1` is acceptable. We round it up to {num_frames}.")

        # Tiler parameters
        tiler_kwargs = {"tiled": tiled, "tile_size": tile_size, "tile_stride": tile_stride}

        # Scheduler
        self.scheduler.set_timesteps(num_inference_steps, denoising_strength=denoising_strength, shift=sigma_shift)
        # Original logic
        noise = self.generate_noise(
            (1, 16, (num_frames - 1) // 4 + 1, height//8, width//8),
            seed=seed, device=rand_device, dtype=torch.float32
        )

        noise = noise.to(dtype=self.torch_dtype, device=self.device)
        if input_video is not None:
            self.load_models_to_device(['vae'])
            input_video = self.preprocess_images(input_video)
            # Remove possible batch dimension
            input_video = [tensor.squeeze(0) if tensor.dim() == 4 else tensor for tensor in input_video]
            input_video = torch.stack(input_video, dim=1)  # [3, T, H, W]
            input_video = input_video.to(dtype=self.torch_dtype, device=self.device)  # [3, T, H, W] - VAE expects this format
            latents = self.encode_video(input_video, **tiler_kwargs).to(dtype=self.torch_dtype, device=self.device)
            latents = self.scheduler.add_noise(latents, noise, timestep=self.scheduler.timesteps[0])
        else:
            latents = noise

        # Encode prompts
        self.load_models_to_device(["text_encoder"])
        prompt_emb_posi = self.encode_prompt(prompt, positive=True)
        if cfg_scale != 1.0:
            prompt_emb_nega = self.encode_prompt(negative_prompt, positive=False)

        # Encode image
        if input_image is not None and self.image_encoder is not None:
            self.load_models_to_device(["image_encoder", "vae"])
            if pseudo_video_path is not None and mask_video_path is not None:
                # New I2V mode
                image_emb = self.encode_image(input_image, pseudo_video_path, mask_video_path)
            else:
                # Original mode (backward compatibility)
                image_emb = self.encode_image(input_image, end_image=end_image, num_frames=num_frames, height=height, width=width)
        else:
            image_emb = {}

        # ControlNet
        if control_video is not None:
            self.load_models_to_device(["image_encoder", "vae"])
            image_emb = self.prepare_controlnet_kwargs(control_video, num_frames, height, width, **image_emb, **tiler_kwargs)

        # Motion Controller
        if self.motion_controller is not None and motion_bucket_id is not None:
            motion_kwargs = self.prepare_motion_bucket_id(motion_bucket_id)
        else:
            motion_kwargs = {}

        # Extra input
        extra_input = self.prepare_extra_input(latents)

        # TeaCache
        tea_cache_posi = {"tea_cache": TeaCache(num_inference_steps, rel_l1_thresh=tea_cache_l1_thresh, model_id=tea_cache_model_id) if tea_cache_l1_thresh is not None else None}
        tea_cache_nega = {"tea_cache": TeaCache(num_inference_steps, rel_l1_thresh=tea_cache_l1_thresh, model_id=tea_cache_model_id) if tea_cache_l1_thresh is not None else None}

        # Unified Sequence Parallel
        usp_kwargs = self.prepare_unified_sequence_parallel()

        # Denoise
        self.load_models_to_device(["dit", "motion_controller"])
        for progress_id, timestep in enumerate(progress_bar_cmd(self.scheduler.timesteps)):
            timestep = timestep.unsqueeze(0).to(dtype=self.torch_dtype, device=self.device)

            # Inference
            noise_pred_posi = model_fn_wan_video(
                self.dit, motion_controller=self.motion_controller,
                x=latents, timestep=timestep,
                **prompt_emb_posi, **image_emb, **extra_input,
                **tea_cache_posi, **usp_kwargs, **motion_kwargs
            )
            if cfg_scale != 1.0:
                noise_pred_nega = model_fn_wan_video(
                    self.dit, motion_controller=self.motion_controller,
                    x=latents, timestep=timestep,
                    **prompt_emb_nega, **image_emb, **extra_input,
                    **tea_cache_nega, **usp_kwargs, **motion_kwargs
                )
                noise_pred = noise_pred_nega + cfg_scale * (noise_pred_posi - noise_pred_nega)
            else:
                noise_pred = noise_pred_posi

            # Scheduler
            latents = self.scheduler.step(noise_pred, self.scheduler.timesteps[progress_id], latents)

        # Decode
        self.load_models_to_device(['vae'])
        frames = self.decode_video(latents, **tiler_kwargs)
        self.load_models_to_device([])
        frames = self.tensor2video(frames[0])

        return frames

class TeaCache:
    def __init__(self, num_inference_steps, rel_l1_thresh, model_id):
        self.num_inference_steps = num_inference_steps
        self.step = 0
        self.accumulated_rel_l1_distance = 0
        self.previous_modulated_input = None
        self.rel_l1_thresh = rel_l1_thresh
        self.previous_residual = None
        self.previous_hidden_states = None

        self.coefficients_dict = {
            "Wan2.1-T2V-1.3B": [-5.21862437e+04, 9.23041404e+03, -5.28275948e+02, 1.36987616e+01, -4.99875664e-02],
            "Wan2.1-T2V-14B": [-3.03318725e+05, 4.90537029e+04, -2.65530556e+03, 5.87365115e+01, -3.15583525e-01],
            "Wan2.1-I2V-14B-480P": [2.57151496e+05, -3.54229917e+04,  1.40286849e+03, -1.35890334e+01, 1.32517977e-01],
            "Wan2.1-I2V-14B-720P": [ 8.10705460e+03,  2.13393892e+03, -3.72934672e+02,  1.66203073e+01, -4.17769401e-02],
        }
        if model_id not in self.coefficients_dict:
            supported_model_ids = ", ".join([i for i in self.coefficients_dict])
            raise ValueError(f"{model_id} is not a supported TeaCache model id. Please choose a valid model id in ({supported_model_ids}).")
        self.coefficients = self.coefficients_dict[model_id]

    def check(self, dit: WanModel, x, t_mod):
        modulated_inp = t_mod.clone()
        if self.step == 0 or self.step == self.num_inference_steps - 1:
            should_calc = True
            self.accumulated_rel_l1_distance = 0
        else:
            coefficients = self.coefficients
            rescale_func = np.poly1d(coefficients)
            self.accumulated_rel_l1_distance += rescale_func(((modulated_inp-self.previous_modulated_input).abs().mean() / self.previous_modulated_input.abs().mean()).cpu().item())
            if self.accumulated_rel_l1_distance < self.rel_l1_thresh:
                should_calc = False
            else:
                should_calc = True
                self.accumulated_rel_l1_distance = 0
        self.previous_modulated_input = modulated_inp
        self.step += 1
        if self.step == self.num_inference_steps:
            self.step = 0
        if should_calc:
            self.previous_hidden_states = x.clone()
        return not should_calc

    def store(self, hidden_states):
        self.previous_residual = hidden_states - self.previous_hidden_states
        self.previous_hidden_states = None

    def update(self, hidden_states):
        hidden_states = hidden_states + self.previous_residual
        return hidden_states



def model_fn_wan_video(
    dit: WanModel,
    motion_controller: WanMotionControllerModel = None,
    x: torch.Tensor = None,
    timestep: torch.Tensor = None,
    context: torch.Tensor = None,
    clip_feature: Optional[torch.Tensor] = None,
    y: Optional[torch.Tensor] = None,
    tea_cache: TeaCache = None,
    use_unified_sequence_parallel: bool = False,
    motion_bucket_id: Optional[torch.Tensor] = None,
    **kwargs,
):
    if use_unified_sequence_parallel:
        import torch.distributed as dist
        from xfuser.core.distributed import (get_sequence_parallel_rank,
                                            get_sequence_parallel_world_size,
                                            get_sp_group)

    t = dit.time_embedding(sinusoidal_embedding_1d(dit.freq_dim, timestep))
    t_mod = dit.time_projection(t).unflatten(1, (6, dit.dim))
    if motion_bucket_id is not None and motion_controller is not None:
        t_mod = t_mod + motion_controller(motion_bucket_id).unflatten(1, (6, dit.dim))
    context = dit.text_embedding(context)

    if dit.has_image_input:
        x = torch.cat([x, y], dim=1)  # (b, c_x + c_y, f, h, w)
        clip_embdding = dit.img_emb(clip_feature)
        context = torch.cat([clip_embdding, context], dim=1)

    x, (f, h, w) = dit.patchify(x)

    freqs = torch.cat([
        dit.freqs[0][:f].view(f, 1, 1, -1).expand(f, h, w, -1),
        dit.freqs[1][:h].view(1, h, 1, -1).expand(f, h, w, -1),
        dit.freqs[2][:w].view(1, 1, w, -1).expand(f, h, w, -1)
    ], dim=-1).reshape(f * h * w, 1, -1).to(x.device)

    # TeaCache
    if tea_cache is not None:
        tea_cache_update = tea_cache.check(dit, x, t_mod)
    else:
        tea_cache_update = False

    # blocks
    if use_unified_sequence_parallel:
        if dist.is_initialized() and dist.get_world_size() > 1:
            x = torch.chunk(x, get_sequence_parallel_world_size(), dim=1)[get_sequence_parallel_rank()]
    if tea_cache_update:
        x = tea_cache.update(x)
    else:
        for block in dit.blocks:
            x = block(x, context, t_mod, freqs)
        if tea_cache is not None:
            tea_cache.store(x)

    x = dit.head(x, t)
    if use_unified_sequence_parallel:
        if dist.is_initialized() and dist.get_world_size() > 1:
            x = get_sp_group().all_gather(x, dim=1)
    x = dit.unpatchify(x, (f, h, w))
    return x

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [12]:
import torch
import os
import glob
import argparse
import re
import random
from PIL import Image
from diffsynth import ModelManager, save_video, VideoData
# from custom_wan_pipe import WanVideoPipeline

# Florence model import - required dependency
from transformers import AutoProcessor, AutoModelForCausalLM

# Global variables to store Florence model
florence_model = None
florence_processor = None

def init_florence_model():
    """Initialize Florence model, only needs to be called once"""
    global florence_model, florence_processor

    if florence_model is not None and florence_processor is not None:
        return True  # Model already loaded, no need to reload

    print("Loading Florence model, please wait...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    # Load model and processor
    florence_model = AutoModelForCausalLM.from_pretrained(
        "multimodalart/Florence-2-large-no-flash-attn", torch_dtype=torch_dtype, trust_remote_code=True
    ).to(device)

    florence_processor = AutoProcessor.from_pretrained(
        "multimodalart/Florence-2-large-no-flash-attn", trust_remote_code=True
    )
    print("Florence model loaded successfully")
    return True

def generate_caption(image, concept_prefix=""):
    """Use Florence model to generate caption for image"""
    global florence_model, florence_processor

    if florence_model is None or florence_processor is None:
        raise RuntimeError("Florence model not initialized, please call init_florence_model() first")

    device = next(florence_model.parameters()).device
    torch_dtype = next(florence_model.parameters()).dtype

    # If input is a path, read image; if PIL Image object, use directly
    if isinstance(image, str):
        image = Image.open(image).convert("RGB")
    elif hasattr(image, 'convert'):
        image = image.convert("RGB")
    else:
        # If it's a numpy array, convert to PIL Image
        if hasattr(image, 'shape'):
            image = Image.fromarray(image).convert("RGB")
        else:
            raise ValueError("Unsupported image format")

    prompt = "<DETAILED_CAPTION>"

    # Construct input
    inputs = florence_processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

    # Generate caption
    generated_ids = florence_model.generate(
        input_ids=inputs["input_ids"], pixel_values=inputs["pixel_values"], max_new_tokens=1024, num_beams=3
    )
    generated_text = florence_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    # Post-processing
    parsed_answer = florence_processor.post_process_generation(
        generated_text, task=prompt, image_size=(image.width, image.height)
    )
    caption_text = parsed_answer["<DETAILED_CAPTION>"].replace("The image shows ", "")

    # Add concept prefix
    if concept_prefix:
        caption_text = f"{concept_prefix} {caption_text}"

    return caption_text

def find_max_epoch_lora(data_dir, use_additional=False):
    """Find the lora file with maximum epoch"""
    lora_dir_name = "lora_additional" if use_additional else "lora"
    lora_base_dir = os.path.join(data_dir, lora_dir_name)
    if not os.path.exists(lora_base_dir):
        if use_additional:
            raise FileNotFoundError(f"Additional LoRA directory does not exist: {lora_base_dir}\n"
                                  f"Please train additional LoRA first using: python train.py --config {os.path.join(data_dir, 'configs', 'training_additional.toml')}")
        else:
            raise FileNotFoundError(f"LoRA directory does not exist: {lora_base_dir}")

    # Find all date directories
    date_dirs = [d for d in os.listdir(lora_base_dir) if os.path.isdir(os.path.join(lora_base_dir, d))]
    if not date_dirs:
        raise FileNotFoundError(f"No training directories found in LoRA directory: {lora_base_dir}")

    # Find the latest training directory (sorted by name, usually datetime format)
    latest_date_dir = sorted(date_dirs)[-1]
    date_dir_path = os.path.join(lora_base_dir, latest_date_dir)

    # Find epoch directories
    epoch_dirs = []
    for item in os.listdir(date_dir_path):
        item_path = os.path.join(date_dir_path, item)
        if os.path.isdir(item_path) and item.startswith("epoch"):
            # Extract epoch number
            match = re.search(r'epoch(\d+)', item)
            if match:
                epoch_num = int(match.group(1))
                epoch_dirs.append((epoch_num, item_path))

    if not epoch_dirs:
        raise FileNotFoundError(f"No epoch directories found in {date_dir_path}")

    # Find maximum epoch
    max_epoch_num, max_epoch_path = max(epoch_dirs, key=lambda x: x[0])
    lora_file_path = os.path.join(max_epoch_path, "adapter_model.safetensors")

    if not os.path.exists(lora_file_path):
        raise FileNotFoundError(f"LoRA file does not exist: {lora_file_path}")

    lora_type = "additional" if use_additional else "standard"
    print(f"Found {lora_type} LoRA file with maximum epoch: epoch{max_epoch_num} - {lora_file_path}")
    return lora_file_path

def find_input_image(data_dir):
    """Find edited image, prioritize png, then jpg"""
    png_path = os.path.join(data_dir, "edited_image.png")
    jpg_path = os.path.join(data_dir, "edited_image.jpg")
    # f"1. Save the edited first frame (from {output_dir}/source_frames/00000.png) as: {output_dir}/edited_image.png (or .jpg)",

    if os.path.exists(png_path):
        print(f"Found edited image: {png_path}")
        return png_path
    elif os.path.exists(jpg_path):
        print(f"Found edited image: {jpg_path}")
        return jpg_path
    else:
        raise FileNotFoundError(f"Edited image does not exist, checked the following paths:\n- {png_path}\n- {jpg_path}")

def validate_paths(model_root_dir, data_dir):
    """Validate if paths exist"""
    if not os.path.exists(model_root_dir):
        raise FileNotFoundError(f"Model root directory does not exist: {model_root_dir}")

    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory does not exist: {data_dir}")

    # Check required model files
    required_files = [
        # "models_clip_open-clip-xlm-roberta-large-vit-huge-14.pth",
        "models_t5_umt5-xxl-enc-bf16.pth",
        "Wan2.1_VAE.pth"
    ]

    for file_name in required_files:
        file_path = os.path.join(model_root_dir, file_name)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Required model file does not exist: {file_path}")

    # Check diffusion model files
    diffusion_model_pattern = os.path.join(model_root_dir, "diffusion_pytorch_model*.safetensors")
    diffusion_model_files = glob.glob(diffusion_model_pattern)
    if not diffusion_model_files:
        raise FileNotFoundError(f"No diffusion model files found, pattern: {diffusion_model_pattern}")

def main(model_root_dir, data_dir, use_additional=False):
    """Main function"""
    try:
        # Validate paths
        print("Validating paths...")
        validate_paths(model_root_dir, data_dir)

        # Infer various paths
        print("Inferring paths...")
        lora_path = find_max_epoch_lora(data_dir, use_additional=use_additional)
        input_image_path = find_input_image(data_dir)
        # pseudo_video_path = os.path.join(data_dir, "inference_rgb.mp4")
        # mask_video_path = os.path.join(data_dir, "inference_mask.mp4")

        # # Check if video files exist
        # if not os.path.exists(pseudo_video_path):
        #     raise FileNotFoundError(f"Pseudo video file does not exist: {pseudo_video_path}")
        # if not os.path.exists(mask_video_path):
        #     raise FileNotFoundError(f"Mask video file does not exist: {mask_video_path}")

        print(f"Using paths:")
        print(f"  Model root directory: {model_root_dir}")
        print(f"  Data directory: {data_dir}")
        print(f"  LoRA file: {lora_path}")
        print(f"  Edited image: {input_image_path}")

        # Automatically find all safetensors files starting with diffusion_pytorch_model
        diffusion_model_pattern = os.path.join(model_root_dir, "diffusion_pytorch_model*.safetensors")
        diffusion_model_files = sorted(glob.glob(diffusion_model_pattern))

        print("Loading models...")
        model_manager = ModelManager(device="cpu")
        # model_manager.load_models([
        #     os.path.join(model_root_dir, "models_clip_open-clip-xlm-roberta-large-vit-huge-14.pth"),
        # ], torch_dtype=torch.float32)
        model_manager.load_models([
            diffusion_model_files,
            os.path.join(model_root_dir, "models_t5_umt5-xxl-enc-bf16.pth"),
            os.path.join(model_root_dir, "Wan2.1_VAE.pth"),
        ], torch_dtype=torch.bfloat16)
        model_manager.load_lora(lora_path, lora_alpha=1.0)
        pipe = WanVideoPipeline.from_model_manager(model_manager, torch_dtype=torch.bfloat16, device="cuda")
        pipe.enable_vram_management(num_persistent_param_in_dit=0)

        # Initialize Florence model
        print("Initializing Florence model...")
        init_florence_model()

        # Load edited image
        input_image = Image.open(input_image_path)

        # Read concept prefix from prefix.txt
        prefix_file = os.path.join(data_dir, 'prefix.txt')
        concept_prefix = ""
        if os.path.exists(prefix_file):
            try:
                with open(prefix_file, 'r', encoding='utf-8') as f:
                    concept_prefix = f.read().strip()
                print(f"Read concept prefix from {prefix_file}: {concept_prefix}")
            except Exception as e:
                print(f"Failed to read prefix.txt file: {e}")
                concept_prefix = "p3rs0n,"  # Use default value
        else:
            print(f"prefix.txt file not found: {prefix_file}, using default prefix")
            concept_prefix = "p3rs0n,"

        # Dynamically generate prompt
        print("Generating caption for edited image...")
        generated_prompt = generate_caption(input_image, concept_prefix=concept_prefix)
        print(f"Generated prompt: {generated_prompt}")

        # Generate random seed
        random_seed = random.randint(0, 2**32 - 1)
        print(f"Using random seed: {random_seed}")

        print("Starting inference...")
        video = pipe(
            prompt=generated_prompt,
            negative_prompt="Overexposure, static, blurred details, subtitles, paintings, pictures, still, overall gray, worst quality, low quality, JPEG compression residue, ugly, mutilated, redundant fingers, poorly painted hands, poorly painted faces, deformed, disfigured, deformed limbs, fused fingers, cluttered background, three legs, a lot of people in the background, upside down",
            input_image=input_image,
            # pseudo_video_path=pseudo_video_path,
            # mask_video_path=mask_video_path,
            num_inference_steps=30,
            seed=random_seed, tiled=True,
            # TeaCache parameters
            tea_cache_l1_thresh=0.275, # The larger this value is, the faster the speed, but the worse the visual quality.
            tea_cache_model_id="Wan2.1-I2V-14B-480P", # Choose one in (Wan2.1-T2V-1.3B, Wan2.1-T2V-14B, Wan2.1-I2V-14B-480P, Wan2.1-I2V-14B-720P).
        )

        output_path = os.path.join(data_dir, "edited_video.mp4")
        save_video(video, output_path, fps=30, quality=5)
        print(f"Video saved to: {output_path}")

    except Exception as e:
        print(f"Error: {e}")
        raise

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description="Video generation inference script")
#     parser.add_argument("--model_root_dir", required=True, help="Model root directory path")
#     parser.add_argument("--data_dir", required=True,    #     default="Wan2.1-T2V-1.3B",
#  help="Data directory path")
#     parser.add_argument("--additional", action="store_true",
#                        help="Use additional LoRA model from lora_additional directory instead of standard lora directory")

#     args = parser.parse_args()
#     main(args.model_root_dir, args.data_dir, use_additional=args.additional) #여기까지 inference.py

# python inference.py --model_root_dir ./Wan2.1-I2V-14B-480P --data_dir ./processed_data/your_sequence
# Wan2.1-T2V-1.3B



In [14]:
# import argparse
# parser = argparse.ArgumentParser(description="Video generation inference script")
# parser.add_argument("--model_root_dir", required=False, default="Wan2.1-T2V-1.3B",
#                     help="Model root directory path")
# parser.add_argument("--data_dir", required=False, default="/content/myfolder/data",
#                     help="Data directory path")
# parser.add_argument("--additional", action="store_true", default="Wan2.1-T2V-1.3B",
#                     help="Use additional LoRA model from lora_additional directory instead of standard lora directory")

# args = parser.parse_args()
# args, unknown = parser.parse_known_args()
# config = ed(vars(args))
from easydict import EasyDict as ed

config = ed({
    "model_root_dir": "/workspace/Wan2.1-T2V-1.3B",
    "data_dir": "/workspace/myfolder/data",
    "additional": False,  # or False
}) #--additional을 cmd에 안적으면 false
# main(config.model_root_dir, config.data_dir, use_additional=args.additional) #여기까지 inference.py
print('done')

done


In [ ]:
# Save your edited first frame as edited_image.png (or .jpg) in the data directory
# Then run inference
python inference.py --model_root_dir ./Wan2.1-I2V-14B-480P --data_dir ./processed_data/your_sequence

In [15]:
!cp /workspace/myfolder/data/00000.jpg /workspace/myfolder/data/edited_image.jpg

In [17]:
main(config.model_root_dir, config.data_dir, use_additional=config.additional)
#lora 폴더를 data폴더 안으로 집어넣기
print('done')

Validating paths...
Inferring paths...
Found standard LoRA file with maximum epoch: epoch10 - /workspace/myfolder/data/lora/20250726_08-49-03/epoch10/adapter_model.safetensors
Found edited image: /workspace/myfolder/data/edited_image.jpg
Using paths:
  Model root directory: /workspace/Wan2.1-T2V-1.3B
  Data directory: /workspace/myfolder/data
  LoRA file: /workspace/myfolder/data/lora/20250726_08-49-03/epoch10/adapter_model.safetensors
  Edited image: /workspace/myfolder/data/edited_image.jpg
Loading models...
Loading models from: ['/workspace/Wan2.1-T2V-1.3B/diffusion_pytorch_model.safetensors']
    model_name: wan_video_dit model_class: WanModel
        This model is initialized with extra kwargs: {'has_image_input': False, 'patch_size': [1, 2, 2], 'in_dim': 16, 'dim': 1536, 'ffn_dim': 8960, 'freq_dim': 256, 'text_dim': 4096, 'out_dim': 16, 'num_heads': 12, 'num_layers': 30, 'eps': 1e-06}
    The following models are loaded: ['wan_video_dit'].
Loading models from: /workspace/Wan2.1-T

config.json: 0.00B [00:00, ?B/s]

configuration_florence2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/multimodalart/Florence-2-large-no-flash-attn:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Florence model loaded successfully
prefix.txt file not found: /workspace/myfolder/data/prefix.txt, using default prefix
Generating caption for edited image...
Generated prompt: p3rs0n, a woman sitting on a couch, surrounded by cushions, using a laptop computer. The background is slightly blurred, giving the focus to the woman and her laptop.
Using random seed: 769003855
Starting inference...


Saving video:   0%|          | 0/81 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Saving video: 100%|██████████| 81/81 [00:00<00:00, 83.63it/s]


Video saved to: /workspace/myfolder/data/edited_video.mp4


In [1]:
main(config.model_root_dir, config.data_dir, use_additional=config.additional)

NameError: name 'main' is not defined

In [18]:
print('done')

done


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import argparse
import json
import decord
from concurrent.futures import ThreadPoolExecutor, as_completed


def sample_video_frames(video_path, num_frames=None):
    vr = decord.VideoReader(video_path)
    total_frames = len(vr)

    if num_frames is None:
        frame_indices = np.arange(total_frames)
    else:
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        frame = vr[int(idx)].asnumpy()
        pil_image = Image.fromarray(frame)
        frames.append(pil_image)

    return frames


def compute_farneback_optical_flow(frames):
    prev_gray = cv2.cvtColor(np.array(frames[0]), cv2.COLOR_BGR2GRAY)
    flow_maps = []
    magnitudes = []
    angles = []
    images = []
    hsv = np.zeros_like(frames[0])
    hsv[..., 1] = 255

    for frame in frames[1:]:
        gray = cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2GRAY)
        flow_map = cv2.calcOpticalFlowFarneback(
            prev_gray,
            gray,
            flow=None,
            pyr_scale=0.5,
            levels=3,
            winsize=15,
            iterations=3,
            poly_n=5,
            poly_sigma=1.2,
            flags=0,
        )
        magnitude, angle = cv2.cartToPolar(flow_map[..., 0], flow_map[..., 1])
        hsv[..., 0] = angle * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        flow_maps.append(flow_map)
        magnitudes.append(magnitude)
        angles.append(angle)
        images.append(bgr)
        prev_gray = gray
    return flow_maps, magnitudes, angles, images


def compute_lk_optical_flow(frames):
    maxCorners = 50
    feature_params = {
        "maxCorners": maxCorners,
        "qualityLevel": 0.3,
        "minDistance": 7,
        "blockSize": 7,
    }
    lk_params = {
        "winSize": (15, 15),
        "maxLevel": 2,
        "criteria": (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    }
    color = np.random.randint(0, 255, (maxCorners, 3))
    old_frame = frames[0]
    old_gray = cv2.cvtColor(np.array(old_frame), cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    mask = np.zeros_like(old_frame)

    for frame in frames[1:]:
        frame_gray = cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(
                mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2
            )
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    return mask


def _downscale_maps(flow_maps, downscale_size: int = 16):
    return [
        cv2.resize(
            flow,
            (downscale_size, int(flow.shape[0] * (downscale_size / flow.shape[1]))),
            interpolation=cv2.INTER_AREA,
        )
        for flow in flow_maps
    ]


def _motion_score(flow_maps):
    average_flow_map = np.mean(np.array(flow_maps), axis=0)
    return np.mean(average_flow_map)


def process_video(video_path):
    frames = sample_video_frames(video_path, num_frames=None)

    farneback, _, _, _ = compute_farneback_optical_flow(frames)
    farneback = float(_motion_score(_downscale_maps(farneback)))
    lucas_kanade = float(_motion_score(compute_lk_optical_flow(frames)))

    return {
        "motion_fb": abs(farneback),
        "motion_lk": lucas_kanade,
    }


# def parse_args():
    # parser = argparse.ArgumentParser()
    # parser.add_argument(
    #     "--input_video_folder",
    #     type=str,
    #     default="demo_result/model_name_input_video",
    # )
    # parser.add_argument(
    #     "--output_json_folder",
    #     type=str,
    #     default="demo_result/model_name_output_json",
    # )
    # parser.add_argument("--num_workers", type=int, default=32)
    # return parser.parse_args()


# def parse_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         "--input_video_folder",
#         type=str,
#         default="content/video",
#     )
#     parser.add_argument(
#         "--output_json_folder",
#         type=str,
#         default="content/output",
#     )
#     parser.add_argument("--num_workers", type=int, default=32)

#     args = parser.parse_args()

#     # EasyDict로 변환
#     config = ed(vars(args))

#     return config
# import argparse
# from easydict import EasyDict as ed

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--input_video_folder",
        type=str,
        default="/content/video",
    )
    parser.add_argument(
        "--output_json_folder",
        type=str,
        default="/content/output",
    )
    parser.add_argument("--num_workers", type=int, default=32)

    args, unknown = parser.parse_known_args()
    config = ed(vars(args))
    return config

# 사용
config = parse_args()
print(config.input_video_folder)

def main():
    args = parse_args()

    input_video_folder = args.input_video_folder
    output_json_folder = args.output_json_folder
    num_workers = args.num_workers

    output_json_file = os.path.join(output_json_folder, "motionscore.json")
    os.makedirs(output_json_folder, exist_ok=True)

    all_results = {}
    video_files = [f for f in os.listdir(input_video_folder) if f.endswith(".mp4")]

    def worker(filename):
        video_path = os.path.join(input_video_folder, filename)
        video_prefix = os.path.splitext(filename)[0]
        motion_scores = process_video(video_path)
        return video_prefix, motion_scores

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(worker, filename) for filename in video_files]
        for future in as_completed(futures):
            video_prefix, motion_scores = future.result()
            all_results[video_prefix] = motion_scores

    with open(output_json_file, "w") as f:
        json.dump(all_results, f, indent=4)

    print(f"All results have been saved to {output_json_file}")


# if __name__ == "__main__":
    # main()

/content/video


In [ ]:
main()

All results have been saved to /content/output/motionscore.json


In [ ]:
  !cp /content/drive/MyDrive/vlora/data/A.mp4 /content/video

In [ ]:


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--input_video_folder",
        type=str,
        default="content/video",
    )
    parser.add_argument(
        "--output_json_folder",
        type=str,
        default="content/output",
    )
    parser.add_argument("--num_workers", type=int, default=32)

    args = parser.parse_args()

    # EasyDict로 변환
    config = ed(vars(args))

    return config

# 사용 예시
config = parse_args()
print(config.input_video_folder)  # 'demo_result/model_name_input_video'
print(config.num_workers)  # 32


usage: colab_kernel_launcher.py [-h] [--input_video_folder INPUT_VIDEO_FOLDER]
                                [--output_json_folder OUTPUT_JSON_FOLDER]
                                [--num_workers NUM_WORKERS]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-54a0da6b-b0ed-42cf-87c4-7a645fdd0d91.json


SystemExit: 2